In [ ]:
!pip3 install segmentation_models_pytorch

In [ ]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
from torch.nn import functional as F

from sklearn.model_selection import train_test_split

import time
from torch.autograd import Variable

import segmentation_models_pytorch as smp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###1.	Необходимо скачать и подготовить датасет https://www.kaggle.com/olekslu/makeup-lips-segmentation-28k-samples/notebooks для обучения модели на сегментацию губ (обратите внимание, что сегментация подразумевает уже два класса: фон и губы).

In [ ]:
photos = []
lips = []
for root, dirs, files in os.walk('/content/drive/MyDrive/set-lipstick-original'):
    for name in files:
        f = os.path.join(root, name)
        if 'photos' in f:
            photos.append(f)
        elif 'lips' in f:
            lips.append(f)
        else:
            break

In [ ]:
len(photos)

28604

In [ ]:
len(lips)

28586

In [ ]:
photos = photos[:5000]
lips = lips[:5000]

In [ ]:
df = pd.DataFrame({'photos': photos, 'lips': lips})
# Отсортируем  датафрейм по значениям
df.sort_values(by='photos',inplace=True)
# Используем функцию,
# лагодаря которой индексация значений 
# будет начинаться с 0.
df.reset_index(drop=True, inplace=True)
# Выведем первые ять значений нашего датафрейма
print(df.head(5))

                                              photos                                               lips
0  /content/drive/MyDrive/set-lipstick-original/p...  /content/drive/MyDrive/set-lipstick-original/l...
1  /content/drive/MyDrive/set-lipstick-original/p...  /content/drive/MyDrive/set-lipstick-original/l...
2  /content/drive/MyDrive/set-lipstick-original/p...  /content/drive/MyDrive/set-lipstick-original/l...
3  /content/drive/MyDrive/set-lipstick-original/p...  /content/drive/MyDrive/set-lipstick-original/l...
4  /content/drive/MyDrive/set-lipstick-original/p...  /content/drive/MyDrive/set-lipstick-original/l...


In [ ]:
class CustomDatasetFromImages(Dataset):
    def __init__(self, data_info):
        self.data_info = data_info
        self.image_arr = self.data_info.iloc[:,0]
        self.label_arr = self.data_info.iloc[:,1]
        self.data_len = len(self.data_info.index)
    def __getitem__(self, index):
        img = np.asarray(Image.open(self.image_arr[index])).astype('float')
        img = torch.as_tensor(img)/255
        img = img.unsqueeze(0).permute(0,3,1,2)
        img = F.interpolate(input=img, size=256, align_corners=False, mode='bicubic')
        
        lab = np.asarray(plt.imread(self.label_arr[index])).astype('float')[:,:,0]*255
        lab = torch.as_tensor(lab).unsqueeze(0)
        lab = lab.unsqueeze(0)
        lab = F.interpolate(input=lab, size=256, mode='nearest')
        
        return (img.float(), lab.float())

    def __len__(self):
        return self.data_len

In [ ]:
# 70 % в тренировочную выборку, 30 - в тестовую
X_train, X_test = train_test_split(df,test_size=0.3)

# Упорядочиваем индексацию
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

# Оборачиваем каждую выборку в наш кастомный датасет
train_data = CustomDatasetFromImages(X_train)
test_data = CustomDatasetFromImages(X_test)

In [ ]:
train_data_loader = DataLoader(train_data,batch_size=8,shuffle=True)
test_data_loader = DataLoader(test_data,batch_size=4,shuffle=False)

In [ ]:
data = next(iter(train_data_loader))
print(data[0].mean(), data[0].std())

tensor(0.6719) tensor(0.2639)


In [ ]:
learning_rate = 0.0001
epochs = 50

###2.	И затем обучить Модель на выбор из segmentation_models_pytorch

In [ ]:
def dice_loss(output, target, weights=None, ignore_index=None):
    """
    output : NxCxHxW Variable
    target :  NxHxW LongTensor
    weights : C FloatTensor
    ignore_index : int index to ignore from loss
    """
    eps = 0.0001

    output = output.float().exp()
    target = target.type(torch.int64)
    encoded_target = output.detach() * 0
    if ignore_index is not None:
        mask = target == ignore_index
        target = target.clone()
        target[mask] = 0
        encoded_target.scatter_(1, target.unsqueeze(1), 1)
        mask = mask.unsqueeze(1).expand_as(encoded_target)
        encoded_target[mask] = 0
    else:
        encoded_target.scatter_(1, target.unsqueeze(1), 1)

    if weights is None:
        weights = 1

    intersection = output * encoded_target
    numerator = 2 * intersection.sum(0).sum(1).sum(1)
    denominator = output + encoded_target

    if ignore_index is not None:
        denominator[mask] = 0
    denominator = denominator.sum(0).sum(1).sum(1) + eps
    loss_per_channel = weights * (1 - (numerator / denominator))

    return loss_per_channel.sum() / output.size(1)

In [ ]:
class SoftDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(SoftDiceLoss, self).__init__()

    def forward(self, logits, targets):
        smooth =1
        num = targets.size(0)
        probs = logits
        m1 = probs.view(num, -1)
        m2 = targets.view(num, -1)
        intersection = (m1 * m2)

        score =(2. * intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        #print(score.sum())
        score =1 - score.sum() / num
        return score
    # This function has only a single output, so it gets only one gradient
    def backward(self, grad_output):

        input, target = self.saved_variables
        grad_input = grad_target = None

        if self.needs_input_grad[0]:
            grad_input = grad_output * 2 * (target * self.union - self.inter) \
                         / (self.union * self.union)
        if self.needs_input_grad[1]:
            grad_target = None

        return grad_input, grad_target

In [ ]:
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
segmodel = smp.Unet('resnet34', classes=2, activation='softmax')
#segmodel = smp.Unet('resnet34', classes=1, activation='softmax').to(device)


In [ ]:
optimizer = torch.optim.Adam(segmodel.parameters(), lr=learning_rate)

In [ ]:
step_losses = []
epoch_losses = []

for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    epoch_loss = []
    t = 0
    time1 = time.time()
    for i, data in enumerate(train_data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        #inputs = inputs.cuda()
        #labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()
  
        # forward + backward + optimize
        outputs = segmodel(inputs[0])
        loss = dice_loss(outputs,labels[0,0,:,:,:])
        loss.backward()
        optimizer.step()
  
        # print statistics
        running_loss += loss.item()
        epoch_loss.append(loss.item())
        t+=1
        if t % 5000 == 4999:    # print every 5000 mini-batches
            print(f'Epoch: {epoch}, batchcount: {t}, avg. loss for last 5000 images: {running_loss/5000}')
    time2 = time.time()
    print(f'Epoch {epoch+1}, loss: ',np.mean(epoch_loss),f' time = {time2-time1} sec')

/usr/local/lib/python3.7/dist-packages/segmentation_models_pytorch/base/modules.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


RuntimeError: ignored